# Day 31 Analysis

Since it's Thanksgiving today in the US, I decided to look at the Thanksgiving Day games I have available in my database and find some fun facts.

In [1]:
import pandas as pd
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

In [3]:
query = """
-- 2 games per season until 2006, then 3 games
WITH thursday_games AS (
    SELECT 
        game_id,
        season,
        week,
        gameday,
        strftime('%d', gameday) as day,
        strftime('%m', gameday) as month,
        gametime,
        away_team,
        away_score,
        home_team,
        home_score,
        result,
        total,
        overtime,
        COUNT() OVER(PARTITION BY home_team) AS num_games
    FROM schedules
    WHERE weekday = 'Thursday'
        AND season <> 2022
    ORDER BY season, week
), thanksgiving_games AS (
    SELECT *
    FROM thursday_games
    WHERE day in ('22', '23', '24', '25', '26', '27', '28') 
        AND month = '11'
        AND season <> 2022
), home_teams AS (
    SELECT
        game_id,
        'home' AS home_away,
        home_team AS team,
        home_score AS score,
        result,
        CASE
            WHEN result > 0 THEN 'win'
            ELSE 'lose'
        END AS win_lose,
        total,
        overtime
    FROM thanksgiving_games
), away_teams AS (
    SELECT
        game_id,
        'away' AS home_away,
        away_team AS team,
        away_score AS score,
        result,
        CASE
            WHEN result < 0 THEN 'win'
            ELSE 'lose'
        END AS win_lose,
        total,
        overtime
    FROM thanksgiving_games
), stacked AS (
    SELECT *
    FROM home_teams
    UNION ALL
    SELECT *
    FROM away_teams
), rankings AS (
    SELECT *,
        COUNT() OVER(PARTITION BY team) AS num_games
    FROM stacked
), summary AS (
    SELECT 
        team,
        home_away,
        win_lose,
        ROUND(AVG(result),2) AS avg_result,
        COUNT(*) AS num_wins_losses,
        ROUND(COUNT(*) * 1.0 / num_games,2) AS pct_win_lose,
        num_games 
    FROM rankings
    GROUP BY team, home_away, win_lose
    ORDER BY num_games DESC, team
)
SELECT *
FROM summary 
"""

pd.read_sql(query, conn).head()

,team,home_away,win_lose,avg_result,num_wins_losses,pct_win_lose,num_games
0,DAL,home,lose,-12.42,12,0.52,23
1,DAL,home,win,14.82,11,0.48,23
2,DET,home,lose,-13.81,16,0.70,23
3,DET,home,win,16.86,7,0.30,23
4,CHI,away,lose,11.67,3,0.43,7


### Fun Facts
1. Detroit has lost 70% of their games
2. The Rams (LA or STL), Browns, and Jags haven't played on Thanksgiving
3. 4 teams have lost by 30+ points: Jets, Eagles, Lions, Packers. Jets & Lions lost by 30+ 2x
4. CHI and GB have 2nd highest # of games played  (7 each) 
5. Titans have the best win: 37pts (against the Lions)
6. 19/61 (~31%) of games were within 1 score (7pts). DAL and CHI have done this 4x
7. 69 pts is the highest total points scored. This has been done 3x